# Aligning output from a Swedish ASR and phonetic recogniser

> "Based only on timestamps"

- toc: false
- branch: master
- badges: false
- comments: true
- hidden: true
- categories: [swedish, asr, phonetic, alignment]

# TODO:

- [X] Add mismatched start time case
- [ ] Fix output to something more CTM-like
- [ ] Read from CTM-merge, take groundtruth vocabulary

In [28]:
def approx_match(time_a, time_b, slippage=(0.01 * 4)):
    return abs(time_a - time_b) <= slippage

In [29]:
from pathlib import Path
TEXT = Path("/Users/joregan/Playing/kbw2v")
PHONES = Path("/Users/joregan/Playing/rd_phonetic")

In [61]:
class Chunk:
    def __init__(self, chunk):
        self.text = chunk['text']
        self.start = chunk['timestamp'][0]
        self.end = chunk['timestamp'][1]
    
    def __repr__(self) -> str:
        return f"[{self.text} ({self.start}, {self.end})]"

class CTMMergeChunk(Chunk):
    def __init__(self, line):
        line_parts = line.split()
        self.start = float(line_parts[2])
        self.duration = float(line_parts[3])
        self.asr_text = line_parts[4]
        self.ref_text = line_parts[6]
        self.operation = line_parts[7]
        self.end = self.start + self.duration
        if self.operation == "cor":
            self.text = self.ref_text
        elif self.operation == "ins":
            self.text = self.asr_text
        else:
            self.text = f"{self.asr_text}/{self.ref_text}"

class SimpleMerge(Chunk):
    def __init__(self, left: Chunk, right: Chunk):
        self.text = left.text
        self.phone = right.text
        self.start = left.start
        self.end = left.end
        self.right_start = right.start
        self.right_end = right.end
        self.diff_start = left.start - right.start
        self.diff_end = left.end - right.end
        if type(left) == CTMMergeChunk:
            self.asr_text = left.asr_text
            self.ref_text = left.asr_text
            self.operation = left.operation
            self.duration = left.duration

    def exact_length(self):
        return self.diff_start == 0 and self.diff_end == 0

    def __repr__(self) -> str:
        return f"[{self.text} :: {self.phone} ({self.start}, {self.end})]"

class ComplexMerge(Chunk):
    def __init__(self, left, right):
        if type(left) == list:
            self.left_chunks = left
        elif left == None:
            self.left_chunks == []
        else:
            self.left_chunks = [left]
        if type(right) == list:
            self.right_chunks = right
        elif right == None:
            self.right_chunks == []
        else:
            self.right_chunks = [right]

        self.start = self.get_start()
        self.end = self.get_end()

        self.text = " ".join([x.text for x in self.left_chunks])
        self.phone = " ".join([x.text for x in self.right_chunks])

    def get_start(self):
        if self.left_chunks == [] and self.right_chunks == []:
            return None
        if self.left_chunks == []:
            return self.right_chunks[0].start
        if self.right_chunks == []:
            return self.left_chunks[0].start
        if self.left_chunks[0].start < self.right_chunks[0].start:
            return self.left_chunks[0].start
        else:
            return self.right_chunks[0].start

    def get_end(self):
        if self.left_chunks == [] and self.right_chunks == []:
            return None
        if self.left_chunks == []:
            return self.right_chunks[-1].end
        if self.right_chunks == []:
            return self.left_chunks[-1].end
        if self.left_chunks[-1].end > self.right_chunks[-1].end:
            return self.left_chunks[-1].end
        else:
            return self.right_chunks[-1].end

    def __repr__(self) -> str:
        return f"[{self.text} :: {self.phone} ({self.start}, {self.end})]"

        

In [31]:
class PhoneChunk(Chunk):
    def __init__(self, chunk):
        self.text = chunk.text
        self.start = chunk.start
        self.end = chunk.end

class WordChunk(Chunk):
    def __init__(self, chunk):
        self.text = chunk.text
        self.start = chunk.start
        self.end = chunk.end
        if type(chunk) == CTMMergeChunk:
            self.asr_text = chunk.asr_text
            self.ref_text = chunk.asr_text
            self.operation = chunk.operation
            self.duration = chunk.duration


In [32]:
def create_merges(text_chunks, phone_chunks):
    merged = []

    pci = 0
    tci = 0

    iteration = 0

    while pci < len(phone_chunks) and tci < len(text_chunks):
        iteration += 1

        am_start = approx_match(text_chunks[tci].start, phone_chunks[pci].start)
        am_end = approx_match(text_chunks[tci].end, phone_chunks[pci].end)

        if am_start and am_end:
            merged.append(SimpleMerge(text_chunks[tci], phone_chunks[pci]))

        elif am_start:
            cur_text = [text_chunks[tci]]
            cur_phone = [phone_chunks[pci]]
            if phone_chunks[pci].end < text_chunks[tci].end:
                while not approx_match(text_chunks[tci].end, phone_chunks[pci].end) and phone_chunks[pci].end < text_chunks[tci].end:
                    pci += 1
                    if pci >= len(phone_chunks):
                        break
                    cur_phone.append(phone_chunks[pci])
                merged.append(ComplexMerge(cur_text, cur_phone))
            else:
                while not approx_match(text_chunks[tci].end, phone_chunks[pci].end) and text_chunks[tci].end < phone_chunks[pci].end:
                    tci += 1
                    if tci >= len(text_chunks):
                        break
                    cur_text.append(text_chunks[tci])
                merged.append(ComplexMerge(cur_text, cur_phone))
        elif am_end:
            cur_text = [text_chunks[tci]]
            cur_phone = [phone_chunks[pci]]
            if text_chunks[tci].start < phone_chunks[pci].start:
                while not approx_match(text_chunks[tci].end, phone_chunks[pci].end) and text_chunks[tci].start < phone_chunks[pci].start:
                    tci += 1
                    if tci >= len(text_chunks):
                        break
                    cur_text.append(text_chunks[tci])
                merged.append(ComplexMerge(cur_text, cur_phone))
            else:
                while not approx_match(text_chunks[tci].end, phone_chunks[pci].end) and phone_chunks[pci].end < text_chunks[tci].end:
                    pci += 1
                    if pci >= len(phone_chunks):
                        break
                    cur_phone.append(phone_chunks[pci])
                merged.append(ComplexMerge(cur_text, cur_phone))
        else:
            if phone_chunks[pci].end < text_chunks[tci].start:
                while phone_chunks[pci].end < text_chunks[tci].start:
                    pci += 1
                    if pci >= len(phone_chunks):
                        break
                    merged.append(PhoneChunk(phone_chunks[pci]))
            elif text_chunks[tci].end < phone_chunks[pci].start:
                while text_chunks[tci].end < phone_chunks[pci].start:
                    tci += 1
                    if tci >= len(text_chunks):
                        break
                    merged.append(WordChunk(text_chunks[tci]))
            else:
                print("else", text_chunks[tci], phone_chunks[pci])
        tci += 1
        pci += 1

    return merged

In [33]:
def print_merges(filename, source, merged):
    with open(filename, "w") as outfile:
        for merge in merged:
            if type(merge) == PhoneChunk:
                outfile.write(f'{source} 1 {merge.start} {merge.end} <UNALIGNED_PHONE> {merge.text.replace(" ", "_")}\n')
            elif type(merge) == WordChunk:
                outfile.write(f'{source} 1 {merge.start} {merge.end} <UNALIGNED_WORD> {merge.text.replace(" ", "_")}\n')
            else:
                outfile.write(f'{source} 1 {merge.start} {merge.end} {merge.text.replace(" ", "_")} {merge.phone.replace(" ", "_")}\n')

In [48]:
import json

API_PATH = Path("/Users/joregan/Playing/rdapi/api_output/")
API_MAPPINGS = {}
for filename in API_PATH.glob("*"):
    api_name = filename.name
    with open(filename) as api_json:
        data = json.load(api_json)
        if "videodata" in data:
            if "streams" in data["videodata"][0] and data["videodata"][0]["streams"] is not None:
                if "files" in data["videodata"][0]["streams"]:
                    if "downloadfileurl" in data["videodata"][0]["streams"]["files"][0]:
                        if data["videodata"][0]["streams"]["files"][0]["downloadfileurl"] != "":
                            video_url = data["videodata"][0]["streams"]["files"][0]["downloadfileurl"]
                            parts = video_url.split("/")
                            API_MAPPINGS[api_name] = parts[-1].replace(".mp4", "")

In [57]:
def ctm_edit_load(filename):
    lines = []
    print(filename)
    with open(filename) as inf:
        for line in inf.readlines():
            lines.append(CTMMergeChunk(line))
    return lines

In [ ]:
import json

CTMEDIT_PATH = Path("/Users/joregan/Playing/rd_ctm_edit/")

for left, right in API_MAPPINGS.items():
    text_chunks = []
    phone_chunks = []

    if not (CTMEDIT_PATH / left).exists():
        continue
    
    text_chunks = ctm_edit_load(str(CTMEDIT_PATH / left))

    with open(str(PHONES / f"{right}.json")) as phones:
        phone_json = json.load(phones)
    for phone_chunk in phone_json['chunks']:
        phone_chunks.append(Chunk(phone_chunk))

    merged = create_merges(text_chunks, phone_chunks)

    print_merges(f"/Users/joregan/Playing/rd_tpalign2/{right}.tsv", right, merged)